# Using XENON fuse

## Imports

In [ ]:
import strax
import straxen
import cutax
import fuse

import numpy as np
from straxen import URLConfig

## Microphysics simulation (former epix)

In fuse all simulation steps are handled in dedicated plugins just like straxen does for our data processing. The first part of our full chain simulations is the simulation of microphysics effects. The corresponding plugins are grouped in the `micro_physics` module.

To set up a simulation we first need to define a `Context`. At this stage we dont't need a lot of inputs so we can run the simulation inside a generic `strax.Context`. Make sure to register all needed plugins and define an `DataDirectory`. Afterwards we can change the configuration of the simulation using `st.set_config`. If you want to try your own `.root` or `.csv`file feel free to change the given example file.

There are some more options we can change: 
- `debug` will print out some debug informations during the simulation (For now there are basically no debug statements.)
- `source_rate`: You can specify a rate in Hz. fuse will distribute your events according to this rate. 
- `n_interactions_per_chunk`: fuse builds strax chunks based on an algorithm called 'dynamic chunking'. It searches for empty time intervalls and cuts the data into chunks there. With this parameter you can set an lower limit to the number of interactions per chunk. For Geant4 files something in the order of 10k - 100k can work. For csv files i would reccomend something in the order of 10 - 1000 depending on you specific data. 
- `cut_delayed`: fuse will remove interactions that happen after the last sampled time + the cut_delayed time. We dont want to deal with interactions that happen way after our "run" finished (yet)
- `DetectorConfigOverride`: Part of the epix detector config - we will update this part at some point. (Here you can set your electric field for the microphysics simulation)

In [ ]:
st = fuse.context.microphysics_context("/path/to/output/folder")

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "debug": True,
               "source_rate": 1,
              })

Now that we have our context we can start to run fuse. Just use `st.make`. Probably there will be a lot of warnings...

In [ ]:
run_number = "00000"

st.make(run_number, "geant4_interactions")
st.make(run_number, "cluster_index")
st.make(run_number, "clustered_interactions")
st.make(run_number, "electric_field_values")
st.make(run_number, "quanta")
st.make(run_number, "microphysics_summary")

If your simulation did not crash you can now load the simulated data just like straxen data. First we can take a look at the `geant4_interaction`. Each line represents a single energy deposit. 

In [ ]:
geant4_interactions = st.get_df("00000",["geant4_interactions", "cluster_index"])
geant4_interactions.head(10)

You can also load the clustered data that will go into the second part of the simulation. For now it is called `microphysics_summary`. Each line gives you now the position and number of electron and photons (along other informations.)

In [ ]:
microphysics_summary = st.get_df("00000",[ "microphysics_summary"])
microphysics_summary.head(10)

## Detector and Electronics Simulation (former WFSim)

Now that you know how to run microphysics simulations we can continue with the detector physics, PMT and DAQ simulations. The corresponding plugins can be found in `detector_physics` and `pmt_and_daq`. 

We want to simulate now up to the raw_records level and then process the data with our usual straxen processing routines. To do so, we will now hijack the simulation context from cutax and register our modules. In the future we aim for a dedicated context where we don't need to do this manualy. Remember to select an output folder with sufficient free disk space as raw_records data can be a bit heavy. 

In [ ]:
url_string = 'simple_load://resource://format://fax_config_nt_sr0_v4.json?&fmt=json'
config = URLConfig.evaluate_dry(url_string)

In [ ]:
st = fuse.context.full_chain_context(out_dir = "/path/to/output/folder",
                                     config = config)

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "debug": True,
               "source_rate": 1,
              })

### Microphysics
Just like before we first run the microphysics simulation. Lets make `microphysics_summary` first. 

In [ ]:
run_number = "00000"

st.make(run_number,"microphysics_summary")

### S1 Simulation

After the microphysics simulation is done we can continue and distribute our S1 photons to our PMTs. The output will be a long list where each row represents a single photon with the information of `time` and `channel`attached.  Strax can show you a progress bar during simulation. I found it to be a bit buggy from times in combination with fuse. 

In [ ]:
st.make(run_number,"propagated_s1_photons" ,progress_bar = True)

### S2 Simulations

The S2 simulations are a bit more complicated than the S1 simulation. We first need to drift our electrons to the liquid-gas interface, extract them and calculate the timing of the electrons when reaching the gas phase. Afterwards we can simulate how many photons each electron generates and then distribute the photons on the PMTs. The output of the S2 simulation has basically the same format as the S1 simulation output. 

In [ ]:
st.make(run_number,"drifted_electrons",progress_bar = True)
st.make(run_number,"extracted_electrons",progress_bar = True)
st.make(run_number,"electron_time",progress_bar = True)
st.make(run_number,"s2_photons",progress_bar = True)
st.make(run_number,"s2_photons_sum",progress_bar = True)
st.make(run_number,"propagated_s2_photons",progress_bar = True)

### PMT Afterpulses

Based on the output of the S1 and S2 simulation we can calculate PMT afterpulses. The output will have the same format as the S1 and S2 simulations where each row represents a "virtual" photon with information about channel and timing attached. 

In [ ]:
st.make(run_number,"pmt_afterpulses",progress_bar = True)

### PMT and DAQ

Combining our S1, S2 and AP simulations we can start to simulate the PMT response to photons and simulate the effects of our DAQ on the PMT output. As a result we get `raw_records` that sould look very similar to the data we get from our real TPC and DAQ! 

In [ ]:
st.make(run_number,"raw_records" , progress_bar = True)

## Processing

We finished our simulation! Now we can use straxen to process it. All necessary straxen plugins should be already registered in the context. 

In [ ]:
st.make(run_number,"event_info")

In [ ]:
event_info = st.get_df(run_number, "event_info")

In [ ]:
event_info.head(20)

## CSV Input

You can also run the simulation without a Geant4 root file but with simulation instructions in a csv file. Simulation instructions from csv files can be passed to fuse on the microphysics level and on the detector physics level.

### Microphysics

First we need to generate some simulation instructions and save them to a csv file. Below you find two examples, one function generating monoenergetic energy deposits from a gamma source and a second function to mimic Kr83m decays.

In [ ]:
import pandas as pd
import numpy as np

def monoenergetic_source(n, energy):
    
    df = pd.DataFrame()
    
    r = np.sqrt(np.random.uniform(0, 2500, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    df['xp'] = r * np.cos(t)
    df['yp'] = r * np.sin(t)
    df['zp'] = np.random.uniform(-150, 0, n)
    
    df['xp_pri'] = df['xp']
    df['yp_pri'] = df['yp']
    df['zp_pri'] = df['zp']
    
    df["ed"] = np.array([energy]*n)
    df["time"] = np.zeros(n)
    df["evtid"] = np.arange(n)
    
    df["type"] = np.repeat("gamma", n)
    
    df["trackid"] =  np.zeros(n)
    df["parentid"] = np.zeros(n, dtype = np.int32)
    df["creaproc"] = np.repeat("None", n)
    df["parenttype"] = np.repeat("None", n)
    df["edproc"] = np.repeat("None", n)
    
    return df

def Kr83m_example(n):
    
    half_life = 156.94e-9 #Kr intermediate state half-life in ns
    decay_energies = [32.2,9.4] # Decay energies in kev
    
    df = pd.DataFrame()
    
    r = np.sqrt(np.random.uniform(0, 2500, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    df['xp'] = np.repeat(r * np.cos(t), 2)
    df['yp'] = np.repeat(r * np.sin(t), 2)
    df['zp'] = np.repeat(np.random.uniform(-150, 0, n), 2)
    
    df['xp_pri'] = df['xp']
    df['yp_pri'] = df['yp']
    df['zp_pri'] = df['zp']
    
    df['ed'] = np.tile(decay_energies,n)
    
    dt = np.random.exponential(half_life/np.log(2),n)
    df['time'] = np.array(list(zip(np.zeros(n), dt))).flatten()#*1e9
    
    df["evtid"] = np.repeat(np.arange(n), 2)
    
    df["parenttype"] = np.tile(['Kr83[41.557]', 'Kr83[9.405]'], n)
    
    #Not used:
    # a) since Kr83m is classified in epix using only the parenttype
    # b) trackid, parentid are not used right now.
    # Please keep in mind that other "simulations" may require properly set 
    # edproc, type and creaproc. Future epix updates using e.g. track reconstructions
    # may also need proper track- and parent-ids. 
    df["trackid"] = np.tile([0,1], n)
    df["parentid"] = np.zeros(2*n, dtype = np.int32)
    df["creaproc"] = np.repeat("None", 2*n)
    df["edproc"] = np.repeat("None", 2*n)
    df["type"] = np.repeat("None", 2*n)

    return df

The time fuse needs to simulate depends on the energy of the signal you put into fuse. The higher the energy the longer fuse needs to simulate. In this example we will simulate 1000 events with an energy of 200 keV. 

In [ ]:
microphysics_instructions = monoenergetic_source(1000, 200)
microphysics_instructions.to_csv("monoenergetic_200keV.csv")

We can now set upt the context like before. fuse will automatically detect that a csv file is given and treat it accordingly. As our csv file contains a single energy deposit per event, we should set `n_interactions_per_chunk` to something reasonable like 100. Now each chunk will consist of at least 100 events. 

In [ ]:
st = fuse.context.full_chain_context(out_dir = "/path/to/output/folder",
                                     config = config)

st.set_config({"path": ".",
               "file_name": "monoenergetic_200keV.csv",
               "debug": True,
               "source_rate": 1,
               "n_interactions_per_chunk": 100,
               "cut_delayed": 4e14,
              })

run_number = "00001"

We do not need to run each simulation step manualy, strax will handle that. Lets just ask it to make `raw_records` and then `event_info`.

In [ ]:
st.make(run_number,"raw_records" , progress_bar = True)

In [ ]:
st.make(run_number,"event_info" , progress_bar = True)

In [ ]:
event_info_data = st.get_df(run_number,"event_info")

Lets end this subsection with an unnecessary complicated scatterplot. 

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

fig = plt.figure()
ax = fig.gca()
ax.set_xlim(1000, 2000)
ax.set_ylim(15000, 60000)

xdata = event_info_data["cs1_wo_timecorr"].values
ydata = event_info_data["cs2_wo_timecorr"].values

xy = np.vstack([xdata, ydata])
z = gaussian_kde(xy)(xy)

ax.scatter(xdata,
           ydata,
           c=z,
           s=7.5,
           cmap="viridis"
          )

ax.set_xlabel("cS1 [pe]")
ax.set_ylabel("cS2 [pe]")

plt.show()

### Detector Physics

Now that we know how to start the microphysics simulation from a csv file we can do the same for the detector physics simulation.
The required csv file format is a bit different. You can find a very simple example below. 

In [ ]:
import pandas as pd
def build_random_instructions(n):

    df = pd.DataFrame()
    
    r = np.sqrt(np.random.uniform(0, 2500, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    df['x'] = r * np.cos(t)
    df['y'] = r * np.sin(t)
    df['z'] = np.random.uniform(-130, -15, n)

    df['photons'] = np.random.uniform(100,5000, n)
    df['electrons'] = np.random.uniform(100,5000, n)
    df["excitons"] = np.zeros(n)

    df['e_field'] = np.array([23]*n)
    df["nestid"] = np.array([7]*n)
    df["ed"] = np.zeros(n)

    #just set the time with respect to the start of the event
    #The events will be distributed in time by fuse
    df["t"] = np.zeros(n)

    df["eventid"] = np.arange(n)

    return df


In [ ]:
detectorphysics_instructions = build_random_instructions(1000)
detectorphysics_instructions.to_csv("random_detectorphysics_instructions.csv",index=False)

We can now set up the simulation context. As we are just running the detector physics simulation we do not need to register the `micro_pyhsics` plugins but need to register the `ChunkCsvInput` plugin.

In [ ]:
st = cutax.contexts.xenonnt_sim_SR0v3_cmt_v9(output_folder = "path/to/output/folder")

detector_simulation_modules = [
    fuse.detector_physics.ChunkCsvInput,
    fuse.detector_physics.S1PhotonPropagation,
    fuse.detector_physics.ElectronDrift,
    fuse.detector_physics.ElectronExtraction,
    fuse.detector_physics.ElectronTiming,
    fuse.detector_physics.SecondaryScintillation,
    fuse.detector_physics.S2PhotonPropagation,
    fuse.pmt_and_daq.PMTAfterPulses,
    fuse.pmt_and_daq.PMTResponseAndDAQ,
    ]


for module in detector_simulation_modules:
    st.register(module)

st.set_config({"input_file": "./random_detectorphysics_instructions.csv",
               "debug": True,
               "source_rate": 1,
               "n_interactions_per_chunk": 25,
              })

run_number = "00002"

In [ ]:
st.make(run_number,"raw_records" , progress_bar = True)

In [ ]:
st.make(run_number,"event_info" , progress_bar = True)

In [ ]:
event_info_data = st.get_df(run_number,"event_info")